In [1]:
import os
import math
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
# =========================
# 1. CSV 병합
# =========================
FILES = [
    "naver_finance_news_2020_prep.csv",
    "naver_finance_news_2021_prep.csv",
    "naver_finance_news_2022_prep.csv",
    "naver_finance_news_2023_prep.csv",
    "naver_finance_news_2024_prep.csv",
]
MERGED_CSV = "naver_finance_news_2020_2024_prep.csv"

df_all = pd.concat([pd.read_csv(f) for f in FILES], ignore_index=True)
df_all = df_all.drop_duplicates().sort_values("date").reset_index(drop=True)
df_all.to_csv(MERGED_CSV, index=False, encoding="utf-8-sig")
print("병합 완료! 최종 행 개수:", len(df_all))

병합 완료! 최종 행 개수: 1715


In [3]:
# =========================
# 2. KoBART 요약
# =========================
INPUT_CSV  = MERGED_CSV
OUTPUT_CSV = "naver_finance_news_2020_2024_with_kobart.csv"
TEXT_COL   = "content"
OUT_COL    = "summary_kobart_v3"

MODEL_NAME = "EbanLee/kobart-summary-v3"

# 요약 파라미터
ENC_MAX_TOKENS   = 1024
GEN_MAX_TOKENS   = 96
NUM_BEAMS        = 5
LENGTH_PENALTY   = 1.2
NO_REPEAT_NGRAM  = 3

# reduce 단계
REDUCE_ENC_MAX   = 768
REDUCE_GEN_MAX   = 120
REDUCE_BEAMS     = 5

# 짧은 본문 배치
BATCH_SIZE_SHORT = 8
SHORT_THRESH     = ENC_MAX_TOKENS

# 데이터 로드
df = pd.read_csv(INPUT_CSV)
texts = df[TEXT_COL].fillna("").astype(str).str.replace("\n", " ").str.strip()
mask = texts.str.len() > 0

# 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device).eval()

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


In [4]:
# -------- Helper --------
def tokenize_len(text: str) -> int:
    return len(tok.encode(text, add_special_tokens=True, truncation=False))

def chunk_by_tokens(text: str, max_tokens: int):
    ids = tok.encode(text, add_special_tokens=False, truncation=False)
    return [tok.decode(ids[i:i+max_tokens], skip_special_tokens=True) 
            for i in range(0, len(ids), max_tokens)]

@torch.no_grad()
def generate_batch(batch_texts, enc_max, gen_max, beams, lp, ngram, device):
    enc = tok(batch_texts, return_tensors="pt", padding=True, truncation=True,
              max_length=enc_max).to(device)
    out = model.generate(
        **enc,
        max_length=gen_max,
        min_length=min(12, gen_max//2),
        num_beams=beams,
        length_penalty=lp,
        no_repeat_ngram_size=ngram,
    )
    return [tok.decode(o, skip_special_tokens=True).replace("\n", " ").strip() for o in out]

def summarize_long(text: str) -> str:
    chunks = chunk_by_tokens(text, ENC_MAX_TOKENS)
    part_sums = generate_batch(chunks, ENC_MAX_TOKENS, GEN_MAX_TOKENS,
                               NUM_BEAMS, LENGTH_PENALTY, NO_REPEAT_NGRAM, device)
    stitched = " ".join(part_sums)
    final = generate_batch([stitched], REDUCE_ENC_MAX, REDUCE_GEN_MAX,
                           REDUCE_BEAMS, LENGTH_PENALTY, NO_REPEAT_NGRAM, device)[0]
    return final

def summarize_short_batch(batch_texts):
    return generate_batch(batch_texts, ENC_MAX_TOKENS, REDUCE_GEN_MAX,
                          NUM_BEAMS, LENGTH_PENALTY, NO_REPEAT_NGRAM, device)

In [5]:
# -------- 실행 --------
summaries = [""] * len(df)
lengths = [tokenize_len(t) if m else 0 for t, m in zip(texts.tolist(), mask.tolist())]

short_idxs = [i for i, l in enumerate(lengths) if mask.iloc[i] and l <= SHORT_THRESH]
long_idxs  = [i for i, l in enumerate(lengths) if mask.iloc[i] and l >  SHORT_THRESH]

# 짧은 본문
for i in tqdm(range(0, len(short_idxs), BATCH_SIZE_SHORT), desc="Summarizing (short)"):
    idx_batch = short_idxs[i:i+BATCH_SIZE_SHORT]
    outs = summarize_short_batch([texts.iloc[k] for k in idx_batch])
    for j, k in enumerate(idx_batch):
        summaries[k] = outs[j]

# 긴 본문
for k in tqdm(long_idxs, desc="Summarizing (long, map-reduce)"):
    summaries[k] = summarize_long(texts.iloc[k])

df[OUT_COL] = summaries
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"요약 완료: {OUTPUT_CSV} (short {len(short_idxs)}건, long {len(long_idxs)}건)")

Summarizing (long, map-reduce): 100%|████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.80s/it]

요약 완료: naver_finance_news_2020_2024_with_kobart.csv (short 1712건, long 3건)


In [6]:
# =========================
# 3. summary 컬럼 정리
# =========================
FINAL_CSV = "naver_finance_news_2020_2024_with_kobart_v2.csv"
df = pd.read_csv(OUTPUT_CSV)

if "summary" in df.columns:
    df = df.drop(columns=["summary"])

df.to_csv(FINAL_CSV, index=False, encoding="utf-8-sig")
print(f"최종 완료: {FINAL_CSV}")

최종 완료: naver_finance_news_2020_2024_with_kobart_v2.csv


In [7]:
df

,date,title,url,content,summary_kobart_v3
0,20200101,올해 원/달러 환율 롤러코스터…연중 변동폭 110원 달해,https://n.news.naver.com/mnews/article/001/001...,올해 원/달러 환율은 롤러코스터를 타는 듯했다. 연중 고점과 저점 차이가 110원 ...,"국내 성장세가 낮아진 가운데 미·중 무역분쟁, 일본 수출규제, 홍콩 시위 등 재료가..."
1,20200101,"위험선호 분위기 속 원/달러 환율 1,150원대로 하락 출발",https://n.news.naver.com/mnews/article/001/001...,30일 오전 원/달러 환율이 하락한 채로 출발했다.이날 서울 외환시장에서 원/달러 ...,"미중 무역합의 낙관론 속 27일 뉴욕증시의 다우존스30 산업평균지수, 스탠더드앤드푸..."
2,20200102,새해 첫 거래일 원/달러 환율 상승 마감,https://n.news.naver.com/mnews/article/001/001...,새해 첫 거래일인 2일 원/달러 환율이 올랐다.이날 서울 외환시장에서 원/달러 환율...,새해 첫 거래일인 2일 서울 외환시장에서 원/달러 환율은 전 거래일보다 1.7원 오...
3,20200103,중동 긴장 고조로 금융시장 불안…주가·환율·금리 '출렁',https://n.news.naver.com/mnews/article/001/001...,미국과 이란의 갈등이 고조되면서 3일 코스피가 1%대의 상승분을 반납하고 원/달러 ...,미국과 이란의 갈등이 고조되면서 3일 코스피가 1%대의 상승분을 반납하고 원/달러 ...
4,20200103,이란군 일인자 美공습 사망에 원/달러 환율 9원 급등,https://n.news.naver.com/mnews/article/001/001...,3일 원/달러 환율이 9원 뛰었다.이란 군부 실세인 거셈 솔레이마니 쿠드스군 사령관...,이란 군부 실세인 거셈 솔레이마니 쿠드스군 사령관이 이라크 바그다드에서 미군 공습에...
...,...,...,...,...,...
1710,20241230,강달러 지속,https://n.news.naver.com/mnews/article/001/001...,"원/달러 환율 1,470원대 중반에서 거래가 이어지고 있는 30일 서울 명동 환전소...","원/달러 환율 1,470원대 중반에서 거래가 이어지고 있는 30일 서울 명동 환전소..."
1711,20241230,달러 강세 계속,https://n.news.naver.com/mnews/article/001/001...,"원/달러 환율 1,470원대 중반에서 거래가 이어지고 있는 30일 서울 명동 환전소...","원/달러 환율 1,470원대 중반에서 거래가 이어지고 있는 30일 서울 명동 환전소..."
1712,20241230,"환율 종가 1,472.5원…연말 기준 외환위기 후 27년 만에 최고",https://n.news.naver.com/mnews/article/001/001...,올해 원/달러 환율 연말 주간 거래 종가가 외환위기였던 1997년 이후 가장 높은 ...,30일 서울 외환시장에서 미국 달러화 대비 원화 환율의 주간 거래 종가는 전 거래일...
1713,20241231,"글로벌 IB들 환율 전망 대폭↑…""내년 3분기까지도 계속 상승""",https://n.news.naver.com/mnews/article/001/001...,금융위기 이후 최고 수준에 달한 원/달러 환율이 새해 들어서도 쉬이 내려가지 않을 ...,금융위기 이후 최고 수준에 달한 원/달러 환율이 새해 들어서도 쉬이 내려가지 않을 ...
